# 네이버 영화평 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')

In [2]:
def auto(model,vect,i):
    g= globals()
    if vect == 'cvect':
        g[f'{vect}{i}'] = CountVectorizer(stop_words='english',ngram_range=(1,i))
    elif vect == 'tvect':
        g[f'{vect}{i}'] = TfidfVectorizer(stop_words='english',ngram_range=(1,i))
    
    g[f'{vect}{i}'].fit(X_train)
    g[f'X_train_{vect}{i}'] = g[f'{vect}{i}'].transform(X_train)
    g[f'X_test_{vect}{i}'] = g[f'{vect}{i}'].transform(X_test)
    model.fit(g[f'X_train_{vect}{i}'],y_train)
    return model.score(g[f'X_test_{vect}{i}'],y_test)

-데이터 가져오기 : '네이버 영화리뷰 데이터'로 검색 

In [3]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",sep='\t')
train_df.shape,test_df.shape,train_df.head(3),train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


((150000, 3),
 (50000, 3),
          id                           document  label
 0   9976970                아 더빙.. 진짜 짜증나네요 목소리      0
 1   3819312  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
 2  10265843                  너무재밓었다그래서보는것을추천한다      0,
 None)

#### 데이터 전처리

- 트레인 데이터 셋

In [4]:
train_df['document'].nunique()

146182

In [5]:
# 중복 여부 확인
train_df['document'].nunique()
# 중복 제거
train_df.drop_duplicates(subset=['document'],inplace=True)
train_df.shape

(146183, 3)

In [6]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [7]:
# Null 데이터 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [8]:
# 부정,긍정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [9]:
## 중복 여부 확인
test_df.document.nunique()

49157

In [10]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [11]:
test_df = test_df.dropna(how='any')
test_df.shape

(49997, 3)

In [12]:
test_df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [53]:
test_df = test_df.dropna(how='any')
test_df.shape

(49726, 3)

In [54]:
test_df.isna().sum()

id          0
document    0
label       0
dtype: int64

### 2. 텍스트 전처리


- Train dataset

In [23]:
# 한글과 공백 이외의 문자는 제거
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ 가-힣 ]','')
train_df

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [51]:
# 데이터가 다 지워져서 ''가 된 데이터 --> np.nan으로 변경한 후 제거
train_df.document.replace('',np.nan,inplace=True)
train_df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [50]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- Test dataset

In [63]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ 가-힣 ]','')
test_df.document.replace('',np.nan,inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape


(49726, 3)

In [64]:
train_df.to_csv('data/naver_movie_train.tsv',sep='\t',index=False)
test_df.to_csv('data/naver_movie_test.tsv',sep='\t',index=False)

# 3. 한글 처리 

In [57]:
from konlpy.tag import Okt
okt = Okt()

In [58]:
text = '흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [32]:
okt.morphs(text,stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [34]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text,stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [40]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 하는 자 와 ㅋㅋ ㅎㅎ ㅠㅠ'
stopwords = set(stop_text.split())
type(stopwords),stopwords

(set,
 {'ㅋㅋ',
  'ㅎㅎ',
  'ㅠㅠ',
  '가',
  '과',
  '는',
  '도',
  '들',
  '를',
  '에',
  '에게',
  '와',
  '으로',
  '은',
  '을',
  '의',
  '이',
  '자',
  '잘',
  '좀',
  '하는'})

In [44]:
from tqdm.notebook import tqdm
X_train = []

for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_train.append(tmp_str)


  0%|          | 0/145791 [00:00<?, ?it/s]

In [65]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/49726 [00:00<?, ?it/s]

In [66]:
y_train = train_df.label.values
y_test = test_df.label.values


4.Feature 변환

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv =cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape,X_test_cv.shape

((145791, 42115), (49726, 42115))

In [69]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv,y_train)
lr.score(X_test_cv,y_test)

0.8248200136749386

#### 6. 실제테스트

In [76]:
review1 = '두시간 반이 왜 이렇게 짧냐'
review2 = '저 정말 스파이더맨 팬입니다. 정말 역겨움을 참느라 힘들었습니다. 이 영화가 재미있다고 하는 사람들 이해할 수가 없네요. '

In [77]:
# 전처리
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ 가-힣 ]','',review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ 가-힣 ]','',review2)
review2

'저 정말 스파이더맨 팬입니다 정말 역겨움을 참느라 힘들었습니다 이 영화가 재미있다고 하는 사람들 이해할 수가 없네요 '

In [78]:
# 불용어 처리
morphs = okt.morphs(review1)
review1 = ' '.join(word for word in morphs if word not in stopwords)
review1

'두 시간 반 왜 이렇게 짧냐'

In [79]:
review2 = ' '.join(word for word in okt.morphs(review2, stem=True) if word not in stopwords)
review2

'저 정말 스파이더맨 팬 이다 정말 역겹다 움 참다 힘들다 영화 재미있다 하다 사람 이해 하다 수가 없다'

In [80]:
##### Feature 변환
review1_cv = cvect.transform([review1])
review2_cv = cvect.transform([review2])


In [82]:
lr.predict(review1_cv),lr.predict(review2_cv)

(array([0], dtype=int64), array([0], dtype=int64))